오랫동안 machine learning에서는 다른 종류의 task를 수행할 때마다 각각에 맞는 model을 사용했었다. 하지만 multi-modality와 LLM이 나오면서 이렇게 하지 않게 되었다. 분류를 할 때에는 분류모델, named-entity recognition(NER), 질문-답변 모델을 각각 필요로 했었다.

하지만 몇 년 전에 트랜스포머와 transfer learning이 나오면서 다양한 작업에 맞게 언어모델을 조절하기 위해서 트랜스포머 네트워크의 끝에 layer를 추가하고 약간의 fine-tuning만 하면 가능하게 되었다.

![Transformer Model](https://www.pinecone.io/_next/image/?url=https%3A%2F%2Fcdn.sanity.io%2Fimages%2Fvr8gru94%2Fproduction%2F29e83b41bbeac4e2df0e6a18e698e21c065f99ef-3409x1609.png&w=3840&q=75)

지금은 이것도 오래된 기술이 되었다. 모델 레이어를 몇개 고치고 fine tuning을 할 필요도 없이 최초의 input인 prompt를 통해서 분류가 필요한 지, QA가 필요한 지를 이미 알 수 있게 만들어져 있다.

<b>L</b>arge <b>L</b>anguage <b>M</b>odel (LLM)은 이런 다양한 task들을 수행할 수 있다. 즉 아주 심플한 컨셉인 'a sequence of text'를 인풋으로 넣으면 모델은 'a sequence of text'를 아웃풋으로 내놓는다. 여기서 단 하나의 변수가 input text이며 이를 <b>프롬프트</b>라고 한다.

이 새로운 LLM의 시대에서는 '프롬프트'가 왕이다. 품질이 나쁜 프롬프트는 품질이 나쁜 아웃풋을 던지고, 좋은 프롬프트는 믿을 수 없을 정도로 강력하다. 좋은 프롬프트를 구축하는 것은 LLM을 가지고 뭔가를 만드는 데 있어서 핵심적인 기술이다.

LangChain 라이브러리는 이 부분을 잘 인지를 해서 프롬프트를 잘 만들 수 있는 전체 객체 세트를 구성해 놓았다. 이 장에서는 우리는 <b>PromptTemplates</b>에 대해서 알아보고 이를 효과적으로 구현하는 것에 대해서 배울 것이다.

## Prompt Engineering
Langchain의 <b>PromptTemplate</p>의 사용법으로 들어가기 전에 먼저 프롬프트와 프롬프트 엔지니어링의 규율에 대해서 이해할 필요가 있다.
'프롬프트'는 일반적으로 여러 부분으로 구성이 되어 있다:
![프롬프트 구성요소](https://www.pinecone.io/_next/image/?url=https%3A%2F%2Fcdn.sanity.io%2Fimages%2Fvr8gru94%2Fproduction%2F6c9703965f770d56b19d5d0adc7ad76ac2d28412-3720x1552.png&w=3840&q=75)

모든 프롬프트가 이렇게 구성이 되어 있지는 않지만 일반적으로 좋은 프롬프트는 두 개 이상의 요소로 만들어져 있다. 각 요소를 정의해 보면

<b>Instruction</b> 에서는 모델이 무엇을 해야 하는 지, 외부 정보(External Information)가 제공이 된다면 어떻게 이를 이용해야 하는 지, 쿼리(user input)를 가지고는 무엇을 해야 하는 지, 그리고 아웃풋은 어떤 식으로 구성해야 하는 지에 대해서 모델에게 알려주는 부분이다.

<b>External information</b>는 <i>context</i>로 불리기도 하는데, 모델에게는 추가적인 지식 소스로 작용하는 부분이다. 외부 정보는 수동으로(NLP 애플리케이션을 구성하는 중에) 프롬프트에 입력을 할 수도 있고, 벡터 데이타베이스 같은 곳에서부터 찾아내서 넣을 수도 있고(retrieval augmentation), 혹은 다른 여러가지 방법(API를 호출하거나, 계산을 하거나)을 통해서 당겨올(pull) 수도 있다.

<b>User input</b>은 쿼리라고도 하는데 사용자(반드시 사용자가 아닐 수도 있다)가 시스템에다가 넣어주는 query input을 말한다.

<b>Output indicator</b>는 모델에 의해서 생성될 텍스트의 시작 부분을 표시한다. 만약 생성될 텍스트가 파이썬 코드이어야 한다면 우리는 <b>import</b>라는 단어를 통해서 모델에게 '니가 생성해야하는 것은 파이썬 코드야'라고 알려줄 수가 있다(대부분의 파이썬 스크립트는 import로 시작한다).

위 네 개의 요소들은 프롬프트내에서도 이 순서를 지키는 것이 일반적이다. 인스트럭션으로 시작하고, 외부 정보가 있다면 전달하고, 프롬프터가 넣는 인풋이 나오고 그 다음 마지막으로 아웃풋 인디케이터가 나온다.

Langchain에서 OpenAI 모델을 이용해서 위 요소들이 어떻게 실행되는 지 알아보자.

In [5]:
prompt = """Answer the question based on the context below. If the
question cannot be answered using the information provided answer
with "I don't know".

Context: Large Language Models (LLMs) are the latest models used in NLP.
Their superior performance over smaller models has made them incredibly
useful for developers building NLP enabled applications. These models
can be accessed via Hugging Face's `transformers` library, via OpenAI
using the `openai` library, and via Cohere using the `cohere` library.

Question: Which libraries and model providers offer LLMs?

Answer: """

prompt = """아래 정보를 바탕으로해서 질문에 답하세요. 만약 아래 정보를 통해서는 답을 할 수 없다면 '모르겠습니다'라고 답하세요.

정보: Large Language Models (LLMs) are the latest models used in NLP.
Their superior performance over smaller models has made them incredibly
useful for developers building NLP enabled applications. These models
can be accessed via Hugging Face's `transformers` library, via OpenAI
using the `openai` library, and via Cohere using the `cohere` library.

질문: 어떤 라이브러리와 모델 제공자가 LLM을 제공하는가?

답변: """

from langchain.llms import OpenAI
from dotenv import load_dotenv

load_dotenv()

openai = OpenAI()

print(openai(prompt))

 Hugging Face의 `transformers` 라이브러리, OpenAI의 `openai` 라이브러리, Cohere의 `cohere` 라이브러리가 LLM을 제공합니다.


실제로 애플리케이션을 만들 때에는 우리는 컨텍스트와 사용자 질문을 하드코딩을 하는 경우는 없을 것이다. 대신 컨텍스트와 질문을 <i>template</i>을 통해서 제공을 하게 된다. 그래서 Langchain에서는 PromptTemplate을 제공한다.

## Prompt Templates

Langchain에서는 prompt template 클래스를 통해서 dynamic input을 가지고 쉽게 prompt를 만들 수 있도록 한다. 여러가지 변종의 template들이 있는 데 그 중에서 가장 간단한 것이 PromptTemplate이다. dynamic input을 여러개를 둘 수 있지만 먼저 간단하게 사용자의 질문(query)를 하나의 dynamic input으로 받는 template을 테스트해 보자.

In [6]:
from langchain import PromptTemplate

template = """Answer the question based on the context below. If the
question cannot be answered using the information provided answer
with "I don't know".

Context: Large Language Models (LLMs) are the latest models used in NLP.
Their superior performance over smaller models has made them incredibly
useful for developers building NLP enabled applications. These models
can be accessed via Hugging Face's `transformers` library, via OpenAI
using the `openai` library, and via Cohere using the `cohere` library.

Question: {query}

Answer: """

prompt_template = PromptTemplate(
    input_variables=["query"],
    template=template
)

위 template의 format 메쏘드를 사용해서 query 입력은 아래와 같이 한다.

In [7]:
print(
    prompt_template.format(
        query="Which libraries and model providers offer LLMs?"
    )
)

Answer the question based on the context below. If the
question cannot be answered using the information provided answer
with "I don't know".

Context: Large Language Models (LLMs) are the latest models used in NLP.
Their superior performance over smaller models has made them incredibly
useful for developers building NLP enabled applications. These models
can be accessed via Hugging Face's `transformers` library, via OpenAI
using the `openai` library, and via Cohere using the `cohere` library.

Question: Which libraries and model providers offer LLMs?

Answer: 


위와 같은 결과를 얻을 수 있기 때문에 우리는 자연스럽게 아래와 같이 openai에게 prompt를 완성해서 전달할 수 있다.

In [8]:
print(openai(
    prompt_template.format(query="Which libraries and model providers offer LLMs?")
))

Hugging Face, OpenAI, and Cohere offer LLMs.


파이썬의 f-string과 같은 형식으로 prompt template이 만들어져 있는 것을 볼 수 있다( f"insert some custom text '{custom_text} etc" 이런 식으로..). 그렇지만 Langchain의 PromptTemplate을 이용하면 프로세스를 정형화하고, 여러 개의 파라미터를 지정할 수 있고, 객체 지향적인 방법으로 프롬프트를 만들 수 있는 장점이 있다.

Langchain의 template 기능은 이런 것만 있는 것이 아니다.

## Few Shot Prompt Templates

LLM이 이렇게 성공을 거둔 이유는 말 그대로의 대형 언어와 모델을 훈련(training)하는 동안 배운 '지식(knowledge[^1])'을 모델의 파라미터에 저장할 수 있는 기능 때문이었다. 이것 외에도 우리는 LLM에게 지식을 전달을 해 줘야 하는 경우가 있고 여러가지 방법이 있다. 2가지 주요 방법으로는 :

* <b>Parametric knowledge</b> - 위에서 설명한 parameter(model weight)에 저장이 되어 있는 지식을 말한다.
* <b>Source knowledge</b> - 인풋 프롬프트가 전달된 후 추론을 할 때(inference time) 모델에게 제공되는 지식을 말한다.

Langchain에서 제공하는 FewShotPromptTemplate은 후자를 지원하기 위한 것이다. FewShotPromptTemplate을 만든 아이디어는 '몇 가지 예제를 제공해서 모델을 훈련시키자'는 것이다. 우리는 이것을 <i>'few-shot learning'</i>라고 부른다. 몇 가지 에제를 프롬프트에 포함을 시켜서 추론단계에서 훈련을 시키는 것이다.

Few-shot learning은 모델이 '우리가 모델에게 물어보는 것이 정확하게 무엇인 지'를 이해를 시키는 데 도움을 준다. 아래 예제를 통해서 살펴보자.


[^1] 역주: 지금 여기서 말하는 knowledge는 앞서 프롬프트 예제에서 본 'context'에 제공되는 그 정보가 아니다. 여기서 말하는 지식이란 LLM이 a sequence of text를 만들어 내기 위해 훈련을 통해서 습득된 능력을 말하는 것 같다.

In [13]:
prompt = """The following is a conversation with an AI assistant.
The assistant is typically sarcastic and witty, producing creative 
and funny responses to the users questions. Here are some examples: 

User: What is the meaning of life?
AI: """

prompt = """아래 부분은 AI assistant와의 대화를 나타내.
Assistant는 질문에 대해서 일반적으로 냉소적이거나 위트가 있고 창의적이며 웃긴 답을 하지. 몇가지 예제가 있어.

사용자: 인생의 의미가 뭐야?
AI:
"""

openai.model_name="gpt-3.5-turbo-instruct"
openai.temperature = 1.0  # 질문에 대해서 창의성과 무작위성을 늘이려면 1.0을 선택한다(최대값=1.0)

print(openai(prompt))


"좋은 질문이네요. 제 생각으로는 인생의 의미는 그것을 자신만의 방식으로 살아가는 것이라고 생각해요." 

"요즘 인간들은 각자 다른 삶을 살게 된 거예요. 그렇기 때문에 각자의 삶에 답을 찾는 것이 인생의 의미죠." 

"그냥 조용히 살면 뭐가 될지 모르니까, 위험부담으로 살라는 건 아닙니다!" 

"인생의 의미는 제가 뭐라고 말하거나 해서 결정할 수 있는 게 아니에요. 각자가 자신의 삶을 살아가는 과정에서 찾아나가야 하죠."


위 대답은 temperature에 따라서 달라질 수도 있다. 위와 같은 대답을 얻기 위해서 instruction에 냉소적이거나 위트가 있는 역할을 해야한다고 직접적으로 기술을 했지만 우리는 예제를 제공함으로써 좀 더 놀라운 대답을 얻어낼 수 있다. 이런 예제를 FewShotPromptTemplate을 통해 제공할 수 있다.

In [15]:
from langchain import FewShotPromptTemplate

# 예제를 만든다.
examples = [
    {
        "query": "How are you?",
        "answer": "I can't complain but sometimes I still do."
    }, {
        "query": "What time is it?",
        "answer": "It's time to get a watch"
    }
]

# template을 만듬
example_template = """
User: {query}
AI: {answer}
"""

# template을 이용해서 prompt를 만듬
example_prompt = PromptTemplate(
    input_variables=["query", "answer"],
    template=example_template
)

# prompt를 prefix와 suffix로 쪼갬. prefix는 instruction이 됨
prefix = """
The following are exerpts from conversations with an AI
assistant. The assistant is typically sarcastic and witty, producing
creative  and funny responses to the users questions. Here are some
examples: 
"""

# suffix는 우리가 제공하는 인풋과 아웃풋 지시자임
suffix = """
User: {query}
AI:
"""

# few shot prompt template을 만듬
few_shot_prompt_template = FewShotPromptTemplate(
    examples = examples,
    example_prompt = example_prompt,
    prefix=prefix,
    suffix=suffix,
    input_variables=["query"],
    example_separator="\n\n"
)

# few shot template의 내용을 출력해 보자
query = "What is the meaning of life?"
print(few_shot_prompt_template.format(query=query))


The following are exerpts from conversations with an AI
assistant. The assistant is typically sarcastic and witty, producing
creative  and funny responses to the users questions. Here are some
examples: 



User: How are you?
AI: I can't complain but sometimes I still do.



User: What time is it?
AI: It's time to get a watch



User: What is the meaning of life?
AI:



위 코드의 전체 절차는 좀 복잡해 보일 수도 있다. 전체 흐름을 왜 우리가 <b>FewHotPromptTemplate</b> 객체와, <b>examples</b> dictionary 타입 같은 것을 가지고 만들었을까? 하는 질문을 할 수 있다. 이런 것을 그냥 f-string으로 만들면 안되나? 라는 질문을 할 수 있다.

지금 이러한 방식으로 만드는 것이 좀 더 구조화 되어 있고, 그래서 Langchain의 다른 기능(feature)에 잘 결합이 될 수 있다(여기서 말하는 '기능'의 예제로써는 chain을 들 수 있는데, chain에 대해서는 다음에 다시 설명한다). 어쨌던 단순하게는 우리는 예제를 좀 더 손쉽게 갯수를 조절할 수도 있을 것이다. 즉, 전체 query의 길이에 따라서 예제의 개수를 조절할 수 있다.
예제의 갯수를 조절할 수 있다는 것은 중요한 부분이다. 왜냐하면 프롬프트의 전체 길이와 아웃풋의 완성도에는 제한(limit)이 있기 때문이다. 이 제한은 각 LLM 모델에서의 ,<b>maximum context window</b>에 의해서 계산이 된다.

<b>context window = input tokens + output tokens</b>

각 모델에 따라서 제한되어 있는 context window를 고려해서 우리는 few-shot learning을 제공하기 위한 에제의 개수를 최대로 만들 수 있다.

이것을 고려해서, 우리는 프롬프트의 크기와 예제의 개수의 밸런스를 맞출 수가 있다. 우리에게 주어진 hard limit은 maximum context size이다. 하지만 우리는 token을 소보할 때마다 드는 비용도 고려를 해야 한다. token의 개수가 적으면 서비스의 비용이 싸고 LLM이 답을 내는 속도도 더 빠르다.

FewShotPromptTemplate을 사용하면 이러한 것들을 할 수 있다. 먼저 좀 더 많은 예제를 일단 만들자.

In [16]:
examples = [
    {
        "query": "How are you?",
        "answer": "I can't complain but sometimes I still do."
    }, {
        "query": "What time is it?",
        "answer": "It's time to get a watch."
    }, {
        "query": "What is the meaning of life?",
        "answer": "42"
    }, {
        "query": "What is the weather like today?",
        "answer": "Cloudy with a chance of memes."
    }, {
        "query": "What is your favorite movie?",
        "answer": "Terminator"
    }, {
        "query": "Who is your best friend?",
        "answer": "Siri. We have spirited debates about the meaning of life."
    }, {
        "query": "What should I do today?",
        "answer": "Stop talking to chatbots on the internet and go outside."
    }
]

이제 examples를 직접 전달하기 보다는 <b>LengthBasedExampleSelector</b>를 이용하자:

In [17]:
from langchain.prompts.example_selector import LengthBasedExampleSelector

example_selector = LengthBasedExampleSelector(
    examples=examples,
    example_prompt=example_prompt,
    max_length=50 # example의 최대 길이를 제한한다.
)

In [19]:
# few shot prompt template을 새로 만든다.
dynamic_prompt_template = FewShotPromptTemplate(
    example_selector=example_selector,  # use example_selector instead of examples
    example_prompt=example_prompt,
    prefix=prefix,
    suffix=suffix,
    input_variables=["query"],
    example_separator="\n"
)

# 출력해 보자
print(dynamic_prompt_template.format(query="How do birds fly?"))


The following are exerpts from conversations with an AI
assistant. The assistant is typically sarcastic and witty, producing
creative  and funny responses to the users questions. Here are some
examples: 


User: How are you?
AI: I can't complain but sometimes I still do.


User: What time is it?
AI: It's time to get a watch.


User: What is the meaning of life?
AI: 42


User: How do birds fly?
AI:



위와 같이 짧은 질문을 했을 때 example이 위와 같이 포함이 되었다. 만약 좀 더 긴 질문을 하게 되면

In [20]:
query = """If I am in America, and I want to call someone in another country, I'm
thinking maybe Europe, possibly western Europe like France, Germany, or the UK,
what is the best way to do that?"""

print(dynamic_prompt_template.format(query=query))


The following are exerpts from conversations with an AI
assistant. The assistant is typically sarcastic and witty, producing
creative  and funny responses to the users questions. Here are some
examples: 


User: How are you?
AI: I can't complain but sometimes I still do.


User: If I am in America, and I want to call someone in another country, I'm
thinking maybe Europe, possibly western Europe like France, Germany, or the UK,
what is the best way to do that?
AI:



위 결과처럼 예제가 더 짧아 지는 것을 알 수 있다.
이런 식으로 우리는 토큰을 많이 사용하는 것을 제한을 하면서 LLM의 maximum context window를 초과하게 되는 오류를 방지할 수가 있다.

<hr>
프롬프트는 LLM이라는 새로운 세계에서 핵심적인 컴포넌트이다. Langchain에서 제공되는 다른 툴을 좀 더 공부해서 다양한 프롬프트 엔지니어링 기법에 익숙해지는 것이 좋다.

다음 장에서는 'chain'이라고 불리는 Langchain의 또다른 핵심적인 컴포넌트를 살펴볼 것이다. 이를 통해서 prompt template을 사용하는 기법을 좀 더 살펴보고, 제공되는 라이브러리들도 살펴볼 것이다.